In [1]:
%load_ext autoreload
%autoreload 2

import dask
dask.config.set(num_workers=20)

import satpy
satpy.config.set({'cache_dir': "D:/sat_data/cache/"})
satpy.config.set({'cache_sensor_angles': True})
satpy.config.set({'cache_lonlats': True})

from dask.diagnostics import Profiler, ResourceProfiler, visualize
from datetime import datetime
from satpy import Scene
from glob import glob
import numpy as np
import os

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

from satpy.utils import debug_on
#debug_on()

In [2]:
import dask
dask.config.set(scheduler='threads') 

tod = 'D:/sat_data/fci/out/'


bbox = (-1600000, -2770000, 690000, -1040000)

comp = 'true_color_flames'

ifiles_l15 = glob('D:/sat_data/fci/in/*FDHSI*')
ifiles_l15.sort()
with dask.config.set({"array.chunk-size": "32MiB"}):
    with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof:
        st = datetime.utcnow()
        #if not os.path.isfile(outf):
        if True:
            scn = Scene(ifiles_l15, reader='fci_l1c_nc')
            scn.load([comp], generate=False)
            scn2 = scn.chunk(chunks=(696*2, 696*2))
           # scn.load(['ir_38'], calibration='radiance')
            #scn.load([comp], generate=False)
            scn3 = scn2.resample(scn2.coarsest_area(), resampler='native')
            scn3.save_dataset(comp, base_dir=tod,  fill_value=0)
            #scn3.save_dataset(comp, base_dir=tod, enhance=False, dtype=np.float32, fill_value=0)

        en = datetime.utcnow()
        print('\n', (en-st).total_seconds())
    visualize([prof, rprof], show=False, save=True, filename="D:/sat_data/ahi_main/frp_new.html")

Don't know how to open the following files: {'D:/sat_data/fci/in\\W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+FCI-1C-RRAD-FDHSI-FD--CHK-TRAIL--DIS-NC4E_C_EUMT_20231120125403_IDPFI_OPE_20231120125007_20231120125924_N_JLS_C_0078_0041.nc', 'D:/sat_data/fci/in\\W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+FCI-1C-RRAD-FDHSI-FD--CHK-TRAIL---NC4E_C_EUMT_20231120125403_IDPFI_OPE_20231120125007_20231120125924_N__C_0078_0041.nc'}



 165.480195
